In [1]:
import json
folder = 'IVAvalid_for_NCTU/day/c32b95371b6b-SOS20230226-143946-2887'
img_json = '/TEST_images.json'
with open(f'{folder}/TEST_images.json') as f:
    img_paths = json.load(f)
label_json = f'{folder}/TEST_objects.json'
with open(label_json) as f:
    label_dicts = json.load(f)

In [ ]:

from torchvision.models import vgg
from torch_lib.Model_heading_bin import *
from torch_lib.ClassAverages import *
from torchvision import transforms
import os, glob, cv2
from library.ron_utils import *
import matplotlib.pyplot as plt
import numpy as np

def plot_regressed_3d_bbox(img, cam_to_img, box_2d, dimensions, alpha, theta_ray, detectionid):

    # the math! returns X, the corners used for constraint
    location, X = calc_location(dimensions, cam_to_img, box_2d, alpha, theta_ray)

    orient = alpha + theta_ray

    #plot_2d_box(img, box_2d, detectionid)
    plot_3d_box(img, cam_to_img, orient, dimensions, location) # 3d boxes

    return location, orient

weights_path = 'weights/DA_B4_N1_50.pkl'
device = torch.device('cpu') # 選gpu的index
checkpoint = torch.load(weights_path, map_location=device) #if training on 2 GPU, mapping on the same device
normalize_type = checkpoint['normal']
bin_num = checkpoint['bin']
angle_per_class = 2*np.pi/float(bin_num)

my_vgg = vgg.vgg19_bn(weights='DEFAULT').to(device)
model = Model(features=my_vgg.features, bins=bin_num).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
# for img processing
if normalize_type == 0:
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
if normalize_type == 1:
    normalize = transforms.Normalize(mean=[0.596, 0.612, 0.587], std=[0.256, 0.254, 0.257])
process = transforms.Compose([transforms.ToTensor(), normalize])


ELAN_averages = ClassAverages(average_file='all_ELAN_class_averages.txt')
cam_to_img = np.array([
        [1.418667e+03, 0.000e+00, 6.4e+02, 0],
        [0.000e+00, 1.418867e+03, 3.6e+02, 0],
        [0.000e+00, 000e+00, 1.0e+00, 0] ])
class_dict = {8:'truck', 9:'car', 10:'motor', 11:'bike'}

# EVAL
model.eval()
for i in range(len(img_paths)):
    path = os.path.join(folder, img_paths[i])
    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    
    label_dict = label_dicts[i]
    for j in range(len(label_dict['labels'])):
        difficulty = label_dict['difficulties'][j] # 1 is hard to distinguish
        if difficulty == 0:
            color = (255, 0, 0)
            line_width = 2
        else:
            color = (0, 0, 255)
            line_width = 1
        class_ = class_dict[label_dict['labels'][j]] # 5:行人,6:機車騎士,7:腳踏車騎士,8:大車,9:小車,10:機車,11:腳踏車
        object_id = label_dict['id'][j] # for tracking
        box_2d = label_dict['boxes'][j]
        box_2d = [[box_2d[0], box_2d[1]], [box_2d[2],box_2d[3]]]
        left_top = box_2d[0]
        right_btm = box_2d[1]
        #cv2.rectangle(img, left_top, right_btm, color, line_width)
        crop = img[left_top[1]:right_btm[1]+1, left_top[0]:right_btm[0]+1] 
        crop = cv2.resize(src = crop, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        crop = process(crop) # expand to 224x224
        crop = torch.stack([crop]).to(device)
        
        [RESIDUALs, BIN_CONFs, delta_DIMs] = model(crop)
        bin_argmax = torch.max(BIN_CONFs, dim=1)[1]
        orient_residual = RESIDUALs[torch.arange(len(RESIDUALs)), bin_argmax] 
        Alphas = angle_per_class*bin_argmax + orient_residual #mapping bin_class and residual to get alpha
        alpha_Elan = float(Alphas[0].data)
        alpha_Elan = angle_correction(alpha_Elan)
        dim_Elan = delta_DIMs.cpu().data.numpy()[0, :]
        dim_Elan += ELAN_averages.get_item(class_)
        
        theta_ray = calc_theta_ray(img.shape[1], box_2d, cam_to_img)
        loc, ry = plot_regressed_3d_bbox(img, cam_to_img, box_2d, dim_Elan, alpha_Elan, theta_ray, j)
        
    plt.imshow(img)
    plt.show()
    
    break

In [ ]:



start = time.time()
for i in range(len(renew_labels)):
    img = cv2.imread(images[i])
    lines = [x.strip() for x in open(renew_labels[i]).readlines()]
    label_ELAN = ''
    for idx, line in enumerate(lines):
        #print(line)
        elements = line.split()
        for j in range(1, len(elements)):
            elements[j] = float(elements[j])

        class_ = elements[0]
        truncate = elements[1]
        occluded = elements[2]

        top_left = (int(round(elements[4])), int(round(elements[5])))
        btm_right = (int(round(elements[6])), int(round(elements[7])))
        box_2d = (top_left, btm_right)
        dim_gt = [elements[8], elements[9], elements[10]] # height, width, length
        crop = img[top_left[1]:btm_right[1]+1, top_left[0]:btm_right[0]+1] 
        crop = cv2.resize(src = crop, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
        crop = process(crop) # expand to 224x224
        #2dbox
        crop = torch.stack([crop]).to(device)
        #Location = [elements[11], elements[12], elements[13]]

        alpha_gt = elements[3]
        ry_gt = elements[14]
        theta_ray = ry_gt - alpha_gt

        [RESIDUALs, BIN_CONFs, delta_DIMs] = model(crop)
        bin_argmax = torch.max(BIN_CONFs, dim=1)[1]
        orient_residual = RESIDUALs[torch.arange(len(RESIDUALs)), bin_argmax] 
        Alphas = angle_per_class*bin_argmax + orient_residual #mapping bin_class and residual to get alpha
        alpha_Elan = float(Alphas[0].data)
        alpha_Elan = angle_correction(alpha_Elan)
        dim_Elan = delta_DIMs.cpu().data.numpy()[0, :]
        dim_Elan += ELAN_averages.get_item(class_)

        loc, ry = plot_regressed_3d_bbox(img, cam_to_img, box_2d, dim_Elan, alpha_Elan, theta_ray, idx)